Code for extracting super-voxel ROIS from the Keller/Zlatic data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import copy
from pathlib import Path

import numpy as np
import pandas as pd
import pyspark

from janelia_core.fileio.exp_reader import read_img_file
from keller_zlatic_vnc.raw_data_processing import video_to_roi_baselines

## Parameters go here

In [3]:
ps = dict()

# Location of excel file specifying where the data for each experiment is saved relative to the base folder
ps['data_loc_file'] = r'A:\projects\keller_vnc\data\EM_volume_experiment_data_locations.xlsx'

# Specify if we use a brain mask
ps['use_brain_mask'] = True

# Specify size of images (if brain mask if not used)
ps['image_size'] = (86, 851, 509)

# Parameters for where the brain mask is located
ps['brain_mask_file'] = r'A:\projects\keller_vnc\data\l1_brain_masks\1099-lsm-ttc-mask-ilastik-20210626-postprocess_brain_only.tiff'

# Name of the subfolder we create under the [Main Folder]\[Subfolder]\ directory to save extracted results
ps['extracted_folder'] = 'extracted\\brain_rois_1_5_5'

# Additional parameters (constant for all datasets) specifying where the data is stored
ps['image_base_folder'] =r'W:\\SV4'
ps['image_processed_folder'] = 'Results\\WeightFused'
ps['image_ext'] = r'weightFused.TimeRegistration.klb'

# Parameters for how we preprocess each image
ps['0_sub_value'] = 100.0

# Parameters for how we extract supervoxels
ps['roi_extract_opts'] = {'voxel_size_per_dim':[1, 5, 5], 'brain_mask_perc': .75}

# Parameters for calculating baseline
ps['baseline_calc_opts'] = {'window_length': 41, 'filter_start': -20, 'write_offset': 20, 'p': .1, 'n_processes': 80}

## Read in excel file specifying location of each dataset

In [4]:
def c_fcn(str):
    return str.replace("'", "")
converters = {0:c_fcn, 1:c_fcn}

data_locs = pd.read_excel(ps['data_loc_file'], header=1, usecols=[1, 2], converters=converters)

## Load the brain mask

In [5]:
if ps['use_brain_mask']:
    brain_mask_file = Path(ps['brain_mask_file'])
    brain_mask = read_img_file(brain_mask_file)
else:
    brain_mask = np.ones(ps['image_size'], dtype='uint16')

## Create a spark Context

In [6]:
conf = pyspark.SparkConf().setMaster('local[20]').setAll([
    ('spark.executor.memory', '10g'), ('spark.driver.memory','400g'), ('spark.driver.maxResultSize', '300g')])
sc = pyspark.SparkContext(conf=conf)

## Process each dataset

In [7]:
def preprocess_f(x):
    x[x==0] = ps['0_sub_value'] 
    return x

In [8]:
roi_extract_opts = copy.copy(ps['roi_extract_opts'])
roi_extract_opts['brain_mask'] = brain_mask
roi_extract_opts['preprocess_f'] = preprocess_f

In [9]:
n_subjects = len(data_locs)
for d_i in range(n_subjects):
    print('**********************************************************************************************')
    print('Processing subject ' + str(d_i + 1) + ' of ' + str(n_subjects))
    
    base_data_dir = Path(ps['image_base_folder']) / data_locs['Main folder'][d_i] / data_locs['Subfolder'][d_i] / ps['image_processed_folder']
    save_dir =  Path(ps['image_base_folder']) / data_locs['Main folder'][d_i] / data_locs['Subfolder'][d_i] / Path(ps['extracted_folder'])
    
    roi_info = video_to_roi_baselines(base_data_dir=base_data_dir, save_dir=save_dir, 
                                             roi_extract_opts=roi_extract_opts, extract_params=ps,
                                             img_file_ext=ps['image_ext'],
                                             baseline_calc_opts=ps['baseline_calc_opts'], new_comp=False, sc=sc)

**********************************************************************************************
Processing subject 1 of 1
Save directory does not already exist.  Creating: W:\SV4\CW_18-02-15\L1-561nm-openLoop_20180215_163233.corrected\extracted\brain_rois_1_5_5
Beginning supervoxel extraction.
Searching for image files...
Found 10367 images.
Extracting: 136544 super voxels from 10367 images.
Extracted 136544 ROIS in 1210.9018800258636 seconds.
Beginning baseline calculation.
Baselines calculated in 1679.777075767517 seconds.
